# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [1]:
pip install requests beautifulsoup4 pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import math

BASE_URL = "http://books.toscrape.com/"

In [5]:
response = requests.get(BASE_URL)
html = response.text

soup = BeautifulSoup(html, 'html.parser')

In [7]:
RATING_MAP = {
    'One': 1,
    'Two': 2,
    'Three': 3,
    'Four': 4,
    'Five': 5
}

def get_genre_links():
    response = requests.get(BASE_URL)
    soup = BeautifulSoup(response.text, 'html.parser')
    genre_tags = soup.select('div.side_categories ul li ul li a')

    links = []
    for tag in genre_tags:
        genre_name = tag.text.strip()
        if genre_name.lower() == 'books':
            continue
        relative_url = tag['href']
        genre_url = BASE_URL + relative_url
        links.append((genre_name, genre_url))
    return links

genre_links = get_genre_links()

In [9]:
def scrape_books(min_rating, max_price):
    all_books = []
    for genre_name, genre_url in genre_links:
        print(f"Processing genre: {genre_name}")
        response = requests.get(genre_url)
        soup = BeautifulSoup(response.text, 'html.parser')

        form_text = soup.find('form', class_='form-horizontal').text
        match = re.search(r'(\d+)\s+results', form_text)
        total_books = int(match.group(1)) if match else 0
        total_pages = math.ceil(total_books / 20)

        print(f"Total books: {total_books}, Total pages: {total_pages}")

        for page_num in range(1, total_pages + 1):
            print(f" -- Processing page {page_num} of {total_pages}")
            if page_num == 1:
                page_url = genre_url
            else:
                page_url = genre_url.replace('index.html', f'page-{page_num}.html')

            response = requests.get(page_url)
            soup = BeautifulSoup(response.text, 'html.parser')

            book_containers = soup.select('article.product_pod')

            for book in book_containers:
                title = book.h3.a['title']

                rating_class = book.find('p', class_='star-rating')['class']
                rating_text = rating_class[1]
                rating = RATING_MAP[rating_text]

                price_text = book.find('p', class_='price_color').text
                price_match = re.search(r'\d+\.\d+', price_text)   
                price = float(price_match.group()) if price_match else None

                if rating < min_rating or (price is not None and price > max_price):
                    continue

                availability = book.find('p', class_='instock availability').text.strip()

                relative_book_url = book.h3.a['href']
                book_url = BASE_URL + 'catalogue/' + relative_book_url.replace('../../', '')

                book_resp = requests.get(book_url)
                book_soup = BeautifulSoup(book_resp.text, 'html.parser')

                upc_tag = book_soup.find('th', string='UPC')
                if upc_tag:
                    upc = upc_tag.find_next_sibling('td').text.strip()
                else:
                    upc = None
                
                description = ''
                desc_heading = book_soup.find('div', id='product_description')
                if desc_heading:
                    desc_p = desc_heading.find_next_sibling('p')
                    if desc_p:
                        description = desc_p.text.strip()
                
                all_books.append({
                    'UPC': upc,
                    'Title': title,
                    'Price (£)': price,
                    'Rating': rating,
                    'Genre': genre_name,
                    'Availability': availability,
                    'Description': description
                })

    print(f"Total books scraped: {len(all_books)}")
    return pd.DataFrame(all_books)

In [11]:
df = scrape_books(4, 20)
df.head()

Processing genre: Travel
Total books: 11, Total pages: 1
 -- Processing page 1 of 1
Processing genre: Mystery
Total books: 32, Total pages: 2
 -- Processing page 1 of 2
 -- Processing page 2 of 2
Processing genre: Historical Fiction
Total books: 26, Total pages: 2
 -- Processing page 1 of 2
 -- Processing page 2 of 2
Processing genre: Sequential Art
Total books: 75, Total pages: 4
 -- Processing page 1 of 4
 -- Processing page 2 of 4
 -- Processing page 3 of 4
 -- Processing page 4 of 4
Processing genre: Classics
Total books: 19, Total pages: 1
 -- Processing page 1 of 1
Processing genre: Philosophy
Total books: 11, Total pages: 1
 -- Processing page 1 of 1
Processing genre: Romance
Total books: 35, Total pages: 2
 -- Processing page 1 of 2
 -- Processing page 2 of 2
Processing genre: Womens Fiction
Total books: 17, Total pages: 1
 -- Processing page 1 of 1
Processing genre: Fiction
Total books: 65, Total pages: 4
 -- Processing page 1 of 4
 -- Processing page 2 of 4
 -- Processing pag

,UPC,Title,Price (£),Rating,Genre,Availability,Description
0,None,The Girl You Lost,12.29,5,Mystery,In stock,
1,None,A Spy's Devotion (The Regency Spies of London #1),16.97,5,Historical Fiction,In stock,
2,None,This One Summer,19.49,4,Sequential Art,In stock,
3,None,"Princess Jellyfish 2-in-1 Omnibus, Vol. 01 (Pr...",13.61,5,Sequential Art,In stock,
4,None,"Outcast, Vol. 1: A Darkness Surrounds Him (Out...",15.44,4,Sequential Art,In stock,


In [13]:
df

,UPC,Title,Price (£),Rating,Genre,Availability,Description
0,None,The Girl You Lost,12.29,5,Mystery,In stock,
1,None,A Spy's Devotion (The Regency Spies of London #1),16.97,5,Historical Fiction,In stock,
2,None,This One Summer,19.49,4,Sequential Art,In stock,
3,None,"Princess Jellyfish 2-in-1 Omnibus, Vol. 01 (Pr...",13.61,5,Sequential Art,In stock,
4,None,"Outcast, Vol. 1: A Darkness Surrounds Him (Out...",15.44,4,Sequential Art,In stock,
...,...,...,...,...,...,...,...
68,None,"Naturally Lean: 125 Nourishing Gluten-Free, Pl...",11.38,5,Food and Drink,In stock,
69,None,The Power Greens Cookbook: 140 Delicious Super...,11.05,5,Food and Drink,In stock,
70,None,Counted With the Stars (Out from Egypt #1),17.97,5,Christian Fiction,In stock,
71,None,The Third Wave: An Entrepreneurâs Vision of ...,12.61,5,Business,In stock,
